## 測試項目
1. 訓練集中的alarm聲
2. 使用pruned and quantized tflite model
4. export to cc file

In [1]:
import os
import sys
import datetime
import shutil
import numpy as np
import tensorflow as tf
# from tensorflow import keras
import zipfile
import wavio
import random
from common import utils as U
np.set_printoptions(threshold=sys.maxsize)

In [5]:
sys.path.append(os.path.abspath("../../src"))
sys.path.append(os.path.abspath("../../src/deployment/"))
sys.path.append(os.path.abspath("../../"))

In [6]:
from lib_original.bytes import to_bytes, from_bytes, byte_conversion_tests, load_data, load_raw, save_raw, save_scores
from lib_original.constants import quant_support, crops, feature_count
# from lib_original.quantize import quantization_tests, get_cast

In [7]:
from datetime import datetime
import re

In [8]:
def quantize_int8(x, axis):
  '''Quantization into int8_t precision, operating on x along axis'''
  scaling_factor_shape = tuple(np.append([len(x)],np.ones(x.ndim - 1, dtype = int)))
  epsilon = 0.000000001
  x_scaling_factor = (np.max(np.abs(x), axis) / 128) + epsilon
  x_scaling_factor = x_scaling_factor.reshape(scaling_factor_shape)
  # x_zero_offset = -0.5 #-0.25 #-0.25
  result = (x / x_scaling_factor) #+ x_zero_offset
  return np.rint(result).astype(np.int8)

In [ ]:
# mask8 = 0x4000 # >> 8 : 16384
# mask7 = 0x2000 # >> 7 :  8192
# mask6 = 0x1000 # >> 6 :  4096
# mask5 = 0x0800 # >> 5 :  2048
# mask4 = 0x0400 # >> 4 :  1024
# mask3 = 0x0200 # >> 3 :   512
# mask2 = 0x0100 # >> 2 :   256
# mask1 = 0x0080 # >> 1 :   128
# mask0 = 0x0040 # >> 0 :    64 below the value, drop the value

In [ ]:
# def maskOP(x):
#     x = np.int16(x)
#     print(f"begin:x:{x}")
#     if (mask8&x):
#         return x >> 8
#     elif (mask7&x):
#         return x >> 7
#     elif (mask6&x):
#         return x >> 6
#     elif (mask5&x):
#         return x >> 5
#     elif (mask4&x):
#         return x >> 4
#     elif (mask3&x):
#         return x >> 3
#     elif (mask2&x):
#         return x >> 2
#     elif (mask1&x):
#         return x >> 1
#     elif (mask0&x):
#         return x
#     else:
#         return 0;

In [ ]:
# def quantize_int8(x, axis):
#     len_of_x = len(x[0][0][0])
#     print(f"len_of_x:{len_of_x}")
#     for i in range(len_of_x):
#         nflag = 2; #positive
#         print("{}:{}".format(i,x[0][0][0][i]))
#         tmp_x = x[0][0][0][i]
#         if tmp_x < 0:
#             tmp_x = np.abs(tmp_x)
#             nflag = 1
#         tmp_x = maskOP(tmp_x)
#         if(nflag==1):
#             tmp_x = -1 * (tmp_x)
#         print("{}:{}".format(i,x[0][0][0][i]))
#         print("*********************************")
#         x[0][0][0][i] = tmp_x
#     return np.rint(x).astype(np.int8)

### Loading TFlite Model

In [9]:
tflite_quant_model_path = "../../trained_models/step_6_QAT_and_Convert2TFLite/final_qat_model_acc87.46_testacc89_20240418190654/qat_model_acc_93_prunInfo_0.8_0.85_testAcc89_20240418190654.tflite";
# tflite_quant_model_path = "./models/tflite/quant_retrained_model_0.85compress_acc_95.4_20240207150608_128.tflite";
# Load quantized TFLite model
tflite_interpreter_quant = tf.lite.Interpreter(model_path=tflite_quant_model_path)

In [10]:
# model.summary()
input_details = tflite_interpreter_quant.get_input_details()
output_details = tflite_interpreter_quant.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

#resize the input and output
# tflite_interpreter_quant.resize_tensor_input(input_details[0]['index'], (2, 1, 30225, 1))
# tflite_interpreter_quant.resize_tensor_input(output_details[0]['index'], (2, 2))

# input_details = tflite_interpreter_quant.get_input_details()
# output_details = tflite_interpreter_quant.get_output_details()

# print("== Input details ==")
# print("name:", input_details[0]['name'])
# print("shape:", input_details[0]['shape'])
# print("type:", input_details[0]['dtype'])

# print("\n== Output details ==")
# print("name:", output_details[0]['name'])
# print("shape:", output_details[0]['shape'])
# print("type:", output_details[0]['dtype'])

tflite_interpreter_quant.allocate_tensors();

== Input details ==
name: input.2_te_transform
shape: [    1     1 30225     1]
type: <class 'numpy.int8'>

== Output details ==
name: Xq.1
shape: [1 3]
type: <class 'numpy.int8'>


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
def genDataTimeStr():
    return datetime.today().strftime('%Y-%m-%d %H:%M:%S').replace('-',"").replace(' ',"").replace(':',"");

In [5]:
def getFileList(srcDir,regex='.*\.wav'):
    results = os.listdir(srcDir)
    out_files = []
    cnt_files = 0
    for file in results:
        if os.path.isdir(os.path.join(srcDir, file)):
            out_files += getFileList(os.path.join(srcDir, file))
        elif re.match(regex, file,  re.I):  # file.startswith(startExtension) or file.endswith(".txt") or file.endswith(endExtension):
            out_files.append(os.path.join(srcDir, file))
            cnt_files = cnt_files + 1
    return out_files

## sound preprocessing functions

In [6]:
_inputLen = 30225
_nCrops = 2
def preprocess_setup():
    funcs = []
    funcs += [U.padding( _inputLen// 2),
              U.normalize(32768.0),
              U.multi_crop(_inputLen, _nCrops)]
              # U.single_crop(_inputLen)]
              # 

    return funcs

def preprocess_debug():
    debug_funcs = []
    debug_funcs += [U.padding( _inputLen// 2),
              # U.normalize(32768.0),]
              U.multi_crop(_inputLen, _nCrops)]
              # U.single_crop(_inputLen)]
              # 

    return debug_funcs


def preprocess(sound, funcs):
    for f in funcs:
        sound = f(sound)
    return sound;
    

# def padding(pad):
#     def f(sound):
#         return np.pad(sound, pad, 'constant')

#     return f
    

# def random_crop(size):
#     def f(sound):
#         org_size = len(sound)
#         start = random.randint(0, org_size - size)
#         return sound[start: start + size]

#     return f

In [7]:
_funcs = preprocess_setup()
# _debug_funcs = preprocess_debug()

### Read Test Wav File

In [8]:
#specify the input type:
dtype = 'int8'

positive_test_wavs = "./test_sounds/positive/"
negative_test_wavs_others = "./test_sounds/negative/othersounds/"
negative_test_wavs_siren = "./test_sounds/negative/siren/"

p_wav_list = getFileList(positive_test_wavs);
n_wav_list_other = getFileList(negative_test_wavs_others);
n_wav_list_siren = getFileList(negative_test_wavs_siren);
print(f"total alarm sounds:{len(p_wav_list)} ");
print(f"total other sounds:{len(n_wav_list_other)}")
print(f"total siren sounds:{len(n_wav_list_siren)}")
p_label = 52;
n_label = 99;

sounds_1 = [];
labels_1 = [];
sounds_2 = [];
labels_2 = [];
## sound = preprocess(sound, _funcs)
for i in p_wav_list:
    sound = wavio.read(i).data.T[0]
    start = sound.nonzero()[0].min();
    end = sound.nonzero()[0].max();
    sound = sound[start: end + 1]
    label = int(p_label);
    if len(sound)> 220500:
        sound = sound[:220500]
    sound = preprocess(sound, _funcs);
    sounds_1.append(sound);
    labels_1.append(label);
    sounds_2.append(sound);
    labels_2.append(label);
    
for j in n_wav_list_other:
    sound = wavio.read(j).data.T[0]
    start = sound.nonzero()[0].min();
    end = sound.nonzero()[0].max();
    sound = sound[start: end + 1];
    label = int(n_label);
    sound = np.int16(preprocess(sound, _funcs));
    # print(sound) # to see the sound is float or int
    sounds_1.append(sound);
    labels_1.append(label);

for k in n_wav_list_siren:
    sound = wavio.read(j).data.T[0]
    start = sound.nonzero()[0].min();
    end = sound.nonzero()[0].max();
    sound = sound[start: end + 1];
    label = int(n_label);
    sound = np.int16(preprocess(sound, _funcs));
    sounds_1.append(sound);
    labels_1.append(label);
    sounds_2.append(sound);
    labels_2.append(label);

print(f"sound testset 1 length: {len(sounds_1)}");
print(f"sound testset 2 length: {len(sounds_2)}");

# sound = wavio.read(test_sound_file).data.T[0]
# start = sound.nonzero()[0].min()
# end = sound.nonzero()[0].max()
# sound = sound[start: end + 1]  # Remove silent sections
# label = 18 #int(os.path.splitext(test_sound_file)[0].split('-')[-1])

# if len(sound)> 220500:
#     sound = sound[:220500]

# ec50_sound1 =  wavio.read(ec50_18_sound).data.T[0]
# start_ec50 = ec50_sound1.nonzero()[0].min()
# end_ec50 = ec50_sound1.nonzero()[0].max()
# ec50_sound1 = ec50_sound1[start_ec50:end_ec50+1]
# ec50_18_label = 18

NameError: name 're' is not defined

In [16]:
_inputLen = 30225
pad = _inputLen// 2
tmp_sound = sounds_1[0];
preprocess(tmp_sound, _funcs)
# tmp_sound = np.pad(sounds_1[0], pad, 'constant');
# tmp_sound = random_crop(tmp_sound,_inputLen)
# tmp_sound = np.expand_dims(s[0], axis=0);
print(tmp_sound[0])
# print(labels)

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

In [17]:
# print()
# print(type(labels_1[0]))

In [18]:
# sound = np.expand_dims(sound, axis=1)
# sound = np.expand_dims(sound, axis=3)
# print(sound.shape)

In [19]:
test1_len = len(sounds_1);
test2_len = len(sounds_2)
len_alarm = len(p_wav_list);
len_siren = len(n_wav_list_siren);
len_other = len(n_wav_list_other);
n_total_len = len_other + len_siren ;
# s_total_len = len(n_wav_list_siren);
print(f"total alarm sounds is {len_alarm}\ntotal other sounds is {n_total_len}(other:{len_other} + siren:{len_siren})");
test1_correct_num = 0;
test2_correct_num = 0;
test1_p_correct_num = 0;
test2_p_correct_num = 0;
n_correct_num = 0;
n_correct_siren_num = 0;
n_correct_othersounds_num = 0;

for w in range(test1_len): 
    s = sounds_1[w]
    l = labels_1[w]
    s_test = np.expand_dims(s[0], axis=0);
    s_test = np.expand_dims(s_test, axis=1);
    s_test = np.expand_dims(s_test, axis=3);
    #get_cast(dtype)(sound, axis = -2)
    # s_test = np.int8(s_test);
    # s_test = get_cast(dtype)(s_test, axis=-2)
    s_test = quantize_int8(s_test,axis=-2)
    
    tflite_interpreter_quant.set_tensor(input_details[0]['index'], s_test);
    tflite_interpreter_quant.invoke()
    pred = tflite_interpreter_quant.get_tensor(output_details[0]['index'])
    # print(f"Prediction result shape:{pred.shape}\n");
    print(f"Prediction result: {pred}, and true label: {l}")
    if l == 52: #positive
        if pred[0][0] > pred[0][1]:
            test1_correct_num += 1;
            test1_p_correct_num += 1;
    if l == 99: #negative
        if pred[0][0] < pred[0][1]:
            test1_correct_num += 1;
            n_correct_num += 1;
print("   ******************   test-1 end   ******************   ")
print("   ******************   test-2 begin (only test domestic alarm and out-door siren)   ******************   ")
for w in range(test2_len): 
    s = sounds_2[w]
    l = labels_2[w]
    s_test = np.expand_dims(s[0], axis=0);
    s_test = np.expand_dims(s_test, axis=1);
    s_test = np.expand_dims(s_test, axis=3);
    # s_test = np.float32(s_test);
    #simple use type cast
    # s_test = np.int8(s_test)
    # s_test = get_cast(dtype)(s_test, axis=-2)
    s_test = quantize_int8(s_test,axis=-2)
    
    # print(s_test.shape)
    # print(f"the {w+1} item's shape:\n  {s.shape}");
    # Run inference
    # print(f"s type:{type(s)}, and s shape:{s.shape}")
    tflite_interpreter_quant.set_tensor(input_details[0]['index'], s_test);
    tflite_interpreter_quant.invoke()
    pred = tflite_interpreter_quant.get_tensor(output_details[0]['index'])
    # print(f"Prediction result shape:{pred.shape}\n");
    print(f"Prediction result: {pred}, and true label: {l}")
    if l == 52: #positive
        if pred[0][0] > pred[0][1]:
            test2_correct_num += 1;
            test2_p_correct_num += 1;
    if l == 99: #negative
        if pred[0][0] < pred[0][1]:
            test2_correct_num += 1;
            n_correct_siren_num += 1;
print(f"test1: total sounds(alarm+other+siren)test result is {100*(test1_correct_num/test1_len)}");
print(f"test1: total alarm sounds result is {100*(test1_p_correct_num/len_alarm)}");
print(f"test1 total other sounds(other+siren) result is {100*n_correct_num/n_total_len}");
print("**************************************************************************************")
print(f"test2: total sounds(alarm+siren)test result is {100*(test2_correct_num/test2_len)}")
print(f"test1: total alarm sounds result is {100*(test2_p_correct_num/len_alarm)}");
print(f"test2 total siren sounds result is {100*n_correct_siren_num/len_siren}");

total alarm sounds is 116
total other sounds is 219(other:128 + siren:91)
Prediction result: [[ 56 -59]], and true label: 52
Prediction result: [[ 45 -48]], and true label: 52
Prediction result: [[ 43 -45]], and true label: 52
Prediction result: [[ 1 -5]], and true label: 52
Prediction result: [[ 47 -50]], and true label: 52
Prediction result: [[ 66 -68]], and true label: 52
Prediction result: [[ 51 -53]], and true label: 52
Prediction result: [[ 37 -40]], and true label: 52
Prediction result: [[  7 -10]], and true label: 52
Prediction result: [[ 35 -38]], and true label: 52
Prediction result: [[ 52 -55]], and true label: 52
Prediction result: [[ 23 -26]], and true label: 52
Prediction result: [[ 29 -33]], and true label: 52
Prediction result: [[ 58 -61]], and true label: 52
Prediction result: [[ 41 -44]], and true label: 52
Prediction result: [[ 68 -70]], and true label: 52
Prediction result: [[ 33 -36]], and true label: 52
Prediction result: [[ 35 -37]], and true label: 52
Prediction

In [20]:
# scores = model.predict(sound, batch_size=len(sound), verbose=0);
# print(type(scores))
# print(scores.shape)

# for res in scores:
#     max_value = res.max()的
#     max_index = np.argmax(res)
#     print(f"max value:{max_value:.5f} and index is {max_index}")
#     print('\n'.join('{}: {:.5f}'.format(*k) for k in enumerate(res)))